<a href="https://colab.research.google.com/github/antoniodragoc/Projeto-de-Graduacao-Antonio-Drago-Caetano/blob/main/2_VGG19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [99]:
# import the needed packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

import matplotlib.image as img

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.metrics import Recall,AUC
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Flatten
from keras.models import Sequential, Model
from keras.layers import Activation, Dense, Dropout, Conv2D

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

from fast_ml.model_development import train_valid_test_split

In [100]:
# Caminhos de treino e validação

# Obs - no diretório dados os dados estão separados em 75% treino 15% validação e 10% teste

path_train = 'D:/Users/N30090/OneDrive - ArcelorMittal/Pictures/Doutorado_img/dados_fetuga/train/'
path_val = 'D:/Users/N30090/OneDrive - ArcelorMittal/Pictures/Doutorado_img/dados_fetuga/val/'

batch_size = 20


In [101]:
# Gerando os dados a partir das imagens
train_datagen = ImageDataGenerator(    
    rotation_range = 90,
    vertical_flip = True,
    horizontal_flip = True,
    zoom_range=0.05,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode = 'reflect',
    samplewise_center = True,
    samplewise_std_normalization = True
    )

val_datagen = ImageDataGenerator(
    samplewise_center = True,
    samplewise_std_normalization = True
    )


test_datagen = ImageDataGenerator(
    samplewise_center = True,
    samplewise_std_normalization = True
    )


In [102]:
# dados de treino
train_generator = train_datagen.flow_from_directory(
    path_train,
    target_size = (256, 256),
    batch_size= batch_size,
    shuffle=True,
    class_mode='categorical' 
)

#dados de validação
val_generator = val_datagen.flow_from_directory(
    path_val,
    target_size = (256, 256),
    batch_size= batch_size,  
    shuffle=True,
    class_mode='categorical'     
)

Found 520 images belonging to 2 classes.
Found 104 images belonging to 2 classes.


In [103]:
y_true = train_generator.classes

In [104]:
val_generator.classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [105]:
# Chamando a rede pré-treinada
conv_model = VGG19(
    weights='imagenet', 
    include_top=False, 
    input_shape=(256,256,3))

first_freeze_layers = 20 #number of the first layers to freeze
for layer in conv_model.layers[0:first_freeze_layers]:
    layer.trainable = False

teste = keras.layers.Flatten()(conv_model.output)

# three hidden layers
teste = keras.layers.Dropout(0.4)(teste)
teste = keras.layers.Dense(128, activation='relu')(teste)
teste = keras.layers.Dense(64, activation='relu')(teste)

# final softmax layer with two categories
predictions = keras.layers.Dense(2, activation='softmax')(teste)

# creating the full model:
full_model = keras.models.Model(inputs=conv_model.input, outputs=predictions)

full_model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0   

In [106]:
#EARLY STOPPING ADICIONADO
# https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/
callback = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=40,
    verbose=0,
    mode="min",
    baseline=None,
    restore_best_weights=True
)

lr_schedule = keras.optimizers.schedules.CosineDecayRestarts(
    initial_learning_rate = 0.001, 
    first_decay_steps = 500, 
    t_mul=2.4, 
    m_mul=1.3, 
    alpha=0.001, 
    name=None
)

In [ ]:
# Compilando o modelo
full_model.compile(loss='categorical_crossentropy',
                  optimizer= keras.optimizers.SGD(learning_rate=lr_schedule),
                  metrics=['accuracy','AUC'])

# Treinando o modelo com os dados de treino e teste
history = full_model.fit(
    train_generator,
    validation_data = val_generator,
    epochs = 100,
    callbacks=[callback]
)

Epoch 1/100
26/26 [==============================] - 59s 2s/step - loss: 0.7494 - accuracy: 0.4923 - auc: 0.4888 - val_loss: 0.7758 - val_accuracy: 0.4135 - val_auc: 0.3107
Epoch 2/100
26/26 [==============================] - 57s 2s/step - loss: 0.6970 - accuracy: 0.5769 - auc: 0.5709 - val_loss: 0.7286 - val_accuracy: 0.4231 - val_auc: 0.4373
Epoch 3/100
26/26 [==============================] - 58s 2s/step - loss: 0.6815 - accuracy: 0.5846 - auc: 0.6044 - val_loss: 0.7247 - val_accuracy: 0.4712 - val_auc: 0.4555
Epoch 4/100
26/26 [==============================] - 57s 2s/step - loss: 0.6917 - accuracy: 0.5365 - auc: 0.5773 - val_loss: 0.7300 - val_accuracy: 0.4904 - val_auc: 0.5175
Epoch 5/100
26/26 [==============================] - 58s 2s/step - loss: 0.6768 - accuracy: 0.5808 - auc: 0.6122 - val_loss: 0.6842 - val_accuracy: 0.5769 - val_auc: 0.5895
Epoch 6/100
26/26 [==============================] - 59s 2s/step - loss: 0.6559 - accuracy: 0.6192 - auc: 0.6593 - val_loss: 0.7001 - v

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# summarize history for auc
plt.plot(history.history['auc'])
plt.plot(history.history['val_auc'])
plt.title('Model auc')
plt.ylabel('auc')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
#dados de teste
path_test = 'D:/Users/N30090/OneDrive - ArcelorMittal/Pictures/Doutorado_img/dados_fetuga/test/'
test_generator =  test_datagen.flow_from_directory(
    path_test,
    target_size = (256, 256),
    shuffle = False,
    batch_size= batch_size,
    class_mode='categorical'     
    )

In [ ]:
#Confution Matrix and Classification Report

from sklearn.metrics import confusion_matrix, classification_report
Y_pred = full_model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)

cmvgg = confusion_matrix(test_generator.classes, y_pred )

In [ ]:
print(classification_report(test_generator.classes, y_pred))

In [ ]:
class_names = ['Benign','Malignant']
report = classification_report(test_generator.classes, y_pred,target_names=class_names, output_dict=True)
df = pd.DataFrame(report).transpose()
df_ = df[['precision', 'recall','f1-score']]
df_ = df_.loc[["Benign", "Malignant","macro avg"]]
df_.to_csv('dados_fetuga_vgg19.csv')
df_.head(10)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns

cm = confusion_matrix(test_generator.classes, y_pred)
#cm = [[24,11],[1,34]]
plt.figure(figsize=(5,5))
x_axis_labels = ['Benign','Malignant'] # labels for x-axis
y_axis_labels = ['Benign','Malignant'] # labels for y-axis
ax = sns.heatmap(cm, annot=True, fmt="d",xticklabels=x_axis_labels, yticklabels=y_axis_labels)
sns.set(font_scale=1.8)
plt.title('Confusion matrix RESNET50')
plt.ylabel('Actual label')
plt.xlabel('Predicted label')